# Naive bayes by hand

In [88]:
import numpy as np
import pandas as pd

Imagine you have 4 apples with these attributes

In [89]:
apples = [
    "red round",
    "red round",
    "green sour round",
    "green round",
]

and 3 bananas with these attributes:

In [90]:
bananas = [
    "yellow skinny",
    "yellow skinny",
    "green skinny"
]

Split into list of lists:

In [91]:
apples = [a.split() for a in apples]
bananas = [b.split() for b in bananas]

**Q.** What is the sorted set of all attributes (assign to vocabulary variable $V$)?

(Let's ignore the unknown word issue in our vectors and in our computations.)

<details>
<summary>Solution</summary>
['green', 'red', 'round', 'skinny', 'sour', 'yellow']
    
You can compute like this:
    
```
Va = set(np.concatenate(apples).ravel())
Vb = set(np.concatenate(bananas).ravel())
V = sorted(Va.union(Vb))
```
</details>

**Q**. What is the word vector for the "red round" apple?

The column values are 1 if the word is mentioned otherwise 0. Assume the sorted column order.

<details>
<summary>Solution</summary>
    The row vector is <tt>[0, 1, 1, 0, 0, 0]</tt> for "red round"
</details>

**Q**. What is the word vector for the "green sour round" apple?

The column values are 1 if the word is mentioned otherwise 0. Assume the sorted column order.

<details>
<summary>Solution</summary>
    The row vector is <tt>[1, 0, 1, 0, 1, 0]</tt> for "green sour round"
</details>

Let's look at all fruit vectors now and fruit target column

In [92]:
data = np.zeros((7,len(V)))
for i,row in enumerate(apples+bananas):
    for w in row:
        data[i,V.index(w)] = 1
df = pd.DataFrame(data,columns=V,dtype=int)
df['fruit'] = [0,0,0,0,1,1,1]
df

,green,red,round,skinny,sour,yellow,fruit
0,0,1,1,0,0,0,0
1,0,1,1,0,0,0,0
2,1,0,1,0,1,0,0
3,1,0,1,0,0,0,0
4,0,0,0,1,0,1,1
5,0,0,0,1,0,1,1
6,1,0,0,1,0,0,1


**Q.** What is a good estimate of $P(apple)$=`P_apple` and $P(banana)$=`P_banana`?

(Define those variables)

<details>
<summary>Solution</summary>
<pre>
P_apple = 4/7
P_banana = 3/7
</pre>
</details>

**Q.** What are good estimates of $P(red|apple)$ and $P(red|banana)$?

Probably best to take ratio of number of apples that are red to the number of apples.  When vector values are binary it feels wrong to do as we did for doc classification.  (In that case, we'd count how many times, say, "red" appears in apple rows and divide by total number of words in apple descriptions. Hmm..I guess same thing as attributes aren't repeated.)

<details>
<summary>Solution</summary>
P(red|apple) = 2/4 apples are red and 0/3 bananas are red.
</details>

**Q.** What are good estimates of $P(green|apple)$ and $P(green|banana)$?

<details>
<summary>Solution</summary>
2/4 apples are green and 1/3 bananas are green.
</details>

## Laplace smoothing of $P(w|c)$

**Q.** If $P(skinny|apple)=0$, what is our smoothed estimate?

<details>
<summary>Solution</summary>
$P(skinny|apple) = (count(skinny,apple)+1)/(count(apple)+|V|) = (0+1)/(4+6) = .1$
</details>

**Now, do that using vector operations to get smoothed `P_w_apple` from the apple records**

Recall that `df[df.fruit==0]` gets you just the apple records.  You should get:

```
green     0.3
red       0.3
round     0.5
skinny    0.1
sour      0.2
yellow    0.1
fruit     0.1
```

<details>
<summary>Solution</summary>
    <tt>P_w_apple = (df[df.fruit==0].sum(axis=0)+1) / (len(apples)+len(V))</tt>
</details>

**Do that same thing to `P_w_banana` from the banana records**

You should get:

```
green     0.222222
red       0.111111
round     0.111111
skinny    0.444444
sour      0.111111
yellow    0.333333
fruit     0.444444
```

<details>
<summary>Solution</summary>
    <tt>P_w_banana = (df[df.fruit==1].sum(axis=0)+1) / (len(bananas)+len(V))</tt>
</details>

**Q.** Given `P_w_apple`, what is `P_apple_redround`, the "probability" that "red round" is an apple?

(We haven't normalized the scores (per our friend Bayes) so they aren't technically probabilities.)  Just compute the score we'd use for classification per the lecture.  Hint: `P_w_apple['skinny']` gives the estimate of $P(skinny|apple)$.

<details>
<summary>Solution</summary>
    The answer is 0.0857142857142857 via:<br>
    <tt>P_apple_redround = P_apple * P_w_apple['red']*P_w_apple['round']</tt>
</details>

**Q.** Given `P_w_banana`, what is `P_banana_redround`, the "probability" that "red round" is a banana?

<details>
<summary>Solution</summary>
    The answer is 0.005291005291005291 via:<br>
    <tt>P_banana_redround = P_banana * P_w_banana['red']*P_w_banana['round']</tt>
</details>

Here's how to easily compute the probability of each word in V given class apple and class banana:

In [ ]:
[P_w_apple[w] for w in V]

In [ ]:
[P_w_banana[w] for w in V]

**Now, define a function for computing likelihood of a document index $d \in [0,5]$**

$$
c^*= \underset{c}{argmax} ~ P(c) \prod_{w \in V} P(w | c)^{n_w(d)}
$$

You have these pieces: `P_apple`, `P_w_apple`, and $n_w(d)$ is just the value in `df[w][d]`.

In [174]:
def likelihood_apple(d:int):
    return ...
def likelihood_banana(d:int):
    return ...

<details>
<summary>Solution</summary>
<pre>
def likelihood_apple(d:int):
    return P_apple * np.product([P_w_apple[w]**df[w][d] for w in V])
def likelihood_banana(d:int):
    return P_banana * np.product([P_w_banana[w]**df[w][d] for w in V])
</pre>
</details>


**Run the following loop to make predictions for each document**

Output should be:

```
0.085714, 0.005291 => apple
0.085714, 0.005291 => apple
0.017143, 0.001176 => apple
0.085714, 0.010582 => apple
0.005714, 0.063492 => banana
0.005714, 0.063492 => banana
0.017143, 0.042328 => banana
```

In [ ]:
for d in range(len(df)):
    a = likelihood_apple(d)
    b = likelihood_banana(d)
    print(f"{a:4f}, {b:4f} => {'apple' if a>b else 'banana'}")